# Callbacks and Tensorboard

We'll go over various callbacks such as:

* Early Stopping
* History (Especially useful for functional models)
* Model History
* Tensorboard
* Model Accuracy and Loss Graphs

In [12]:
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras import Model, layers
from tensorflow.keras.models import load_model, model_from_json
tensorflow.keras.__version__

'2.2.4-tf'

In [2]:
train_datagen = ImageDataGenerator(
    shear_range=10,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    preprocessing_function=preprocess_input)

In [3]:
train_generator = train_datagen.flow_from_directory(
    'data/train',
    batch_size=32,
    class_mode='categorical',
    target_size=(224, 224))

Found 12894 images belonging to 10 classes.


In [4]:
validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
validation_generator = validation_datagen.flow_from_directory(
    'data/val',
    shuffle=False,
    class_mode='categorical',
    target_size=(224, 224))

Found 500 images belonging to 10 classes.


In [5]:
conv_base = ResNet50(include_top=False, weights='imagenet')

/home/chris/.miniconda/envs/tf2/lib/python3.7/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [6]:
# Unfreeze some layers
for layer in conv_base.layers[:-4]:
    layer.trainable = True

In [7]:
#model.summary()

In [8]:
x  = conv_base.output
x = layers.GlobalAveragePooling2D()(x)

predictions = layers.Dense(10, activation='softmax', name='fc3')(x)

In [9]:
model = Model(conv_base.input, predictions)

In [11]:
opt = tensorflow.keras.optimizers.Adam()
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

## Callbacks

Callbacks are a good way to:

* get feedback about training while training
* save history for analysis
* checkpoint save your model whenever performance increases

You can use Tensorboard for advanced diagnostics and graph view

You will need this command for Tensorboard: `logdir=graph/ --host 0.0.0.0 --port 8999`

In [14]:
filepath = 'models/weights.{epoch:02d}-{val_loss:.2f}.hdf5'

stopping_criterion = keras.callbacks.EarlyStopping(monitor='val_acc', patience=2)

history = keras.callbacks.History()

checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True)

tensorboard = keras.callbacks.TensorBoard(log_dir='graph', histogram_freq=0, write_graph=True)

callbacks = [stopping_criterion, history, checkpoint, tensorboard]

In [ ]:
history